# 中心軸の上下端点を指定するツール

### **Usage**

```main(file = "自動計測データ.xlsx", target=10)```

マウスで２点をクリックし、ENTERで確定してください。 
「R」でやり直しできます。

In [155]:
import numpy as np
import cv2
import pandas as pd
import os
from rdlib2 import *
import matplotlib.pyplot as plt
# %matplotlib inline


# 重心の位置を求める   (rdlib2にある関数）
'''def getCoG(img):
    _lnum, _img, cnt, cog = cv2.connectedComponentsWithStats(img)
    areamax = np.argmax(cnt[1:,4])+1 # ０番を除く面積最大値のインデックス
    c_x,c_y = np.round(cog[areamax]) # 重心の位置を丸めて答える
    x,y,w,h = cnt[areamax,0:4] # 囲む矩形の x0,y0,w,h
    return c_x,c_y,x,y,w,h
'''
UNIT = 256

def main1(target=10,file="計測指示＆記録.xlsx"):
    global rdimg,sx,sy,rdcolor,rdcnt,uppercnt,lowercnt,c_x,c_y,x_r,y_r,radius,gcounter,tflag,bflag
    df = pd.read_excel(file)
    gcounter = 0
    
    def initcondition():
        global gcounter,radius,rdcolor,tflag,bflag
        tflag,bflag = False,False # 上下の点が決まっているかどうかのフラグ
        radius = 10
        gcounter += 1
        rdcolor = rdcolor = cv2.cvtColor(rdimg, cv2.COLOR_GRAY2BGR)
    
    for radish in range(len(df)):
        idata = df.iloc[radish]
        topdir = idata['topdir']  #  画像ファイルのパスのベース
        subdir = idata['subdir']  #  サブディレクトリ
        filename = idata['filename'] #  ファイル名
        check = idata['処理対象'] #  処理対象かどうかのフラグ　　test がTrueの時のみ意味がある
        if check !=target : #  check が 10 でない画像はスルーする
                continue

        path = os.path.join(topdir,subdir,filename)
        print("処理対象画像 {}\n".format(path))
        src= cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        org= cv2.imread(os.path.join('一軍',subdir,filename),1) 
        org = cv2.resize(org,(int(org.shape[1]/2),int(org.shape[0]/2)))
        
        rdimg = getstandardShape(src, unitSize=UNIT, thres=0.25, setrotation = 0, norotation = True)
        _img,contours,hierarchy = cv2.findContours(rdimg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
        rdcnt = contours[np.argmax([len(c) for c in contours])] # 輪郭線情報　global 変数　
        
        c_x,c_y,x_r,y_r,width,height = getCoG(rdimg) # 重心の位置、バウンダリボックスの左上(x,y), 幅と高さ
        
        uppercnt = [[x,y] for [[x,y]] in rdcnt if y >= y_r and y < y_r + int(height/4)]
        lowercnt = [[x,y] for [[x,y]] in rdcnt if y >= y_r+int(0.75*height) ]
        
        print(len(uppercnt),len(lowercnt))
        x0,y0,w0,h0 = cv2.boundingRect(rdcnt) # 外接矩形
        # print("x0 {} y0 {} width {} height {}".format(x0,y0,width,height))

        mu = cv2.moments(rdimg, False)
        gx,gy= int(mu["m10"]/mu["m00"]) , int(mu["m01"]/mu["m00"]) # 重心

        initcondition()       
        while True:
                wkimg = rdcolor.copy()
                cv2.namedWindow("PLOT")
                cv2.namedWindow(filename)
                cv2.setMouseCallback("PLOT", onMouse)
                cv2.imshow("PLOT", wkimg)
                cv2.imshow(filename,org)
                cv2.moveWindow(filename, 0, 500)
                key = cv2.waitKey(0)
                if key==ord('r') or key == ord('R'):
                    initcondition()
                elif key==2: # LEFT 
                    radius = radius - 1  if radius > 1 else 1
                    
                elif key==3: # RIGHT
                    radius = radius + 1

                elif key==0: # UP
                    radius = radius + 3

                elif key==1: # DOWN
                    radius = radius - 3 if radius > 1 else 2

                elif key==ord('r'):
                    initcondition()
                elif key==13: # Enter で確定　次へ,
                    if not(tflag and bflag):
                        continue
                    font = cv2.FONT_HERSHEY_PLAIN
                    print("確定しますが? ENTER->確定  R -> やり直し")
                    cv2.putText(rdcolor,"OK?",(10,60),font,1,(0,255,0))
                    cv2.putText(rdcolor,"Ent->Record",(10,75),font,1,(0,255,0))
                    cv2.putText(rdcolor,"R->Reset",(10,90),font,1,(0,255,0))
                    key2 = cv2.waitKey(0)
                    if key2 == 13:
                        df.loc[radish,'TOPX'] = topdx
                        df.loc[radish,'TOPY'] = topdy
                        df.loc[radish,'TOPDR'] = topdr
                        df.loc[radish,'BTMX'] = btmdx
                        df.loc[radish,'BTMY'] = btmdy
                        df.loc[radish,'BTMDR'] = btmdr
                        df.loc[radish,'処理対象'] = 1
                        df.to_excel(file, index=True, header=True)
                        print("確定しました\n\n")
                        break
                    elif key2 == ord('r') or key2 == ord('R'):
                        initcondition()
                elif key==27: # ESC で終了
                    break
                [cx,cy],_d = nearestPos(sx,sy)
                drawcircle(wkimg)
        if key==27: # ESC で終了
            cv2.destroyAllWindows()
            break
    cv2.waitKey(1)
    
# (x,y)に最も近い輪郭上の点を答える
def nearestPos(x,y):
    # global  rdcnt,uppercnt,lowercnt,c_x,c_y,x_r,y_r
    cnt = uppercnt if y <= c_y else lowercnt
    diff = [[x-cx,y-cy] for [cx,cy] in cnt] # 輪郭点と(x,y)を結ぶベクトル
    distance = [np.sqrt(dx*dx+dy*dy) for dx,dy in diff] # ベクトルの長さ = 距離
    mi = np.argmin(distance)
    return cnt[mi],distance[mi]

def drawcircle(wkimg):
    global sx,sy
    [cx,cy],d_ = nearestPos(sx,sy)
    cv2.circle(wkimg,(cx,cy),5,(0,0,255),-3)
    cv2.circle(wkimg,(cx,cy),radius,(255,0,255),2)
    cv2.imshow("PLOT", wkimg)
    
def drawcursor(wkimg):
    [h,w] = wkimg.shape[:2]
    cv2.line(wkimg,(sx,0),(sx,h),(255,0,0),1)
    cv2.line(wkimg,(0,sy),(w,sy),(255,0,0),1)       

# マウスのコールバック関数　マウスイベントに対する応答
def onMouse(event, x, y, flags,params):    
    global rdimg,sx,sy,rdcolor,radius,sx,sy,tflag,bflag,topdx,topdy,topdr,btmdx,btmdy,btmdr
    sx,sy = x,y
    wkimg = rdcolor.copy()
    # クリックされた時
    if event == cv2.EVENT_LBUTTONUP:
        sx, sy = x, y # マウスの位置をグローバル変数に代入して処理ルーチンに伝える
        [cx,cy] ,distance= nearestPos(x,y)
        print("({},{}) - ({},{}) distance {}".format(x,y,cx,cy,distance))
        if y <= c_y and tflag == False: # 上端確定
            topdx,topdy,topdr = cx,cy,radius
            tflag = True
        elif y > c_y and bflag == False:
            btmdx,btmdy,btmdr = cx,cy,radius
            bflag = True
        else:
            return
        cv2.circle(rdcolor,(cx,cy),radius,(0,0,255),-1)
        cv2.circle(rdcolor,(cx,cy),3,(255,255,255),-1)
    # マウスが移動ている間は十字カーソルを表示
    if event == cv2.EVENT_MOUSEMOVE:
        wkimg = rdcolor.copy()
        drawcursor(wkimg)
        drawcircle(wkimg)

In [157]:
main1(file="計測指示＆記録.xlsx",target=8)
cv2.destroyAllWindows()
cv2.waitKey(1)

処理対象画像 シルエット/17Cylindric/17daruma3o02_l.jpg

154 146
(198,60) - (198,64) distance 4.0
(188,320) - (185,314) distance 6.708203932499369
確定しますが? ENTER->確定  R -> やり直し
確定しました


処理対象画像 シルエット/17Cylindric/17daruma3o03_l.jpg

143 138
(196,311) - (194,298) distance 13.152946437965905
(180,64) - (180,66) distance 2.0
確定しますが? ENTER->確定  R -> やり直し
確定しました


処理対象画像 シルエット/17Cylindric/17daruma3o05_l.jpg

150 129
(204,58) - (204,65) distance 7.0
(159,316) - (159,313) distance 3.0
確定しますが? ENTER->確定  R -> やり直し
確定しました


処理対象画像 シルエット/17Elliptic/17ofukur2o09_l.jpg

150 134
(197,362) - (197,319) distance 43.0
(191,53) - (191,64) distance 11.0
確定しますが? ENTER->確定  R -> やり直し
確定しました


処理対象画像 シルエット/17Elliptic/17ofukur3o01_l.jpg

136 146
(187,335) - (193,316) distance 19.924858845171276
(206,62) - (205,64) distance 2.23606797749979
(187,335) - (193,316) distance 19.924858845171276
確定しますが? ENTER->確定  R -> やり直し
確定しました


処理対象画像 シルエット/17Elliptic/17ofukur3o02_l.jpg

129 126
(192,57) - (192,66) distance 9.0
(187,309) - (

-1

# 

In [137]:
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [148]:
round(0.4)

0